In [1]:

import numpy as np
import cooler


In [2]:

# Author: Tong liu
# 08/30/2022


In [6]:
##### get testing input data


# time point ids
ids = ["PN5","early_2cell","late_2cell","8cell","ICM","mESC_500"]

chrs_test = ['chr6']
resolution = 40000
sub_mat_n = int(2_000_000 / resolution)
dir_data = "./data/"
num_bins_all = []

for chr in range(1,21):

    chrid = 'chr'+str(chr)
    if chr == 20:
        chrid = 'chrX'
    if chrid not in chrs_test:
        continue

    dat_timePoints = []
    dat_index = []
    for idx, timePoint in enumerate(ids):
 
        ficool = dir_data+"cool_40kb_downsample/"+timePoint + ".cool"
        clr = cooler.Cooler(ficool)
   
        chr_len = clr.chromsizes[chrid]
        mat_chr = clr.matrix(balance=False).fetch(chrid)
        mat_chr2 = np.nan_to_num(mat_chr)
        
        bins = mat_chr2.shape[0]
        if idx == 0:
            num_bins_all.append(bins)
        subMats = []
        index = []
        #print(chrid, chr_len, mat.shape)
        for i in range(0, bins, 3):
            if i+sub_mat_n >= bins:
                continue
            subMat = mat_chr2[i:i+sub_mat_n, i:i+sub_mat_n]
            subMats.append(subMat)
            index.append((i, i))
        
        subMats = np.array(subMats)
        index = np.array(index)
        dat_timePoints.append(subMats)
        dat_index.append(index)
        
            
    dat_timePoints = np.array(dat_timePoints)
    dat_timePoints2 = np.transpose(dat_timePoints, (1,0,2,3))
    dat_index = np.array(dat_index)
    dat_index2 = np.transpose(dat_index, (1,0,2))
    
    print(chrid, dat_timePoints.shape, dat_timePoints2.shape, dat_index.shape, dat_index2.shape)
    np.save(dir_data+"data_test_"+chrid, dat_timePoints2)
    np.save(dir_data+"data_test_index_"+chrid, dat_index2)
    

chr6 (6, 1230, 50, 50) (1230, 6, 50, 50) (6, 1230, 2) (1230, 6, 2)


In [ ]:

##### run the following command for predicting Hi-C of the last three time steps

# sh HiC4D_predict.sh



In [ ]:

##### combine the predicted submatrices to get the final big matrix


In [11]:

def get_predictions(file_predict, file_index, num_bins, sub_mat_n=50):
    
    dat_predict = np.load(file_predict)
    dat_index = np.load(file_index)
    
    predictions = []
    for i in range(-3,0):
        tid = "t"+str(i+7)
        mat_chr = np.zeros((num_bins, num_bins))
        mat_n = np.zeros((num_bins, num_bins))
        for j in range(dat_predict.shape[0]):
            i1, i2 = dat_index[j, i]
            
            mat_chr[i1:(i1+sub_mat_n), i2:(i2+sub_mat_n)] += dat_predict[j, i]
            mat_n[i1:(i1+sub_mat_n), i2:(i2+sub_mat_n)] += 1
     
        mat_chr2 = np.divide(mat_chr, mat_n, out=np.zeros_like(mat_chr), where=mat_n!=0)
        predictions.append(mat_chr2)
    
    return predictions


In [13]:

file_predict = "./data/chr6_predicted.npy"
file_index = "./data/data_test_index_chr6.npy"
file_out = "./data/chr6_predicted_final"

predict_mat = get_predictions(file_predict, file_index, num_bins_all[0])

# for t4, t5, and t6
print(np.array(predict_mat).shape)
np.save(file_out, np.array(predict_mat))

(3, 3738, 3738)
